# Library Import

In [ ]:
# ----- BASE IMPORT -----
import pandas as pd
import numpy as np
import csv,nltk
from numpy import array
import string
import random
from math import floor
from collections import OrderedDict
from collections import defaultdict

# ----- NATURAL LANGUAGE IMPORT -----
!python -m nltk.downloader all
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import  pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from operator import itemgetter, attrgetter

# ----- MACHINE LEARNING IMPORT -----
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import NearestCentroid
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

/opt/conda/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /usr/share/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /usr/share/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloadi

# Functions Definition

## Dataset Managment Functions

In [ ]:
def dataset_creation():
    disease_symptom_dict = defaultdict(list)
    with open('../input/ontobasedclassification/Final_Dataset.csv','r',encoding='utf-8') as csvFile:
      reader=csv.reader(csvFile)
      data=list(reader)
    csvFile.close()
    keywords=[]
    # print(len(data))
    for i in range(1,len(data)):
      # print(i,'Entity:'+data[i][0])
      data[i][0].replace('\xa0',' ')
      # print('Keywords:'+data[i][1])
      # print('Additional Keywords:'+data[i][2])
      for j in range(0,len(data[i][1])):
        data[i][1][j].replace('\xa0',' ')
      keywords=data[i][1].split(',')
      # print(keywords)
      disease_symptom_dict[str(data[i][0]).strip()].append(keywords)
    # print(len(disease_symptom_dict))
    x = []
    y = []
    for i in disease_symptom_dict:
        y.append(i)
        x.append(disease_symptom_dict[i][0])
    temp = []
    for i in x:
        for j in i:
            temp.append(j)

    from collections import OrderedDict
    temp = list(OrderedDict((x, True) for x in temp).keys())
    for i in range(len(y)):
      for j in range(150):
          y.append(y[i])
          x.append(random.sample(x[i],floor(len(x[i])/3)))
    x_ = []
    for i in x:
      x_.append(' '.join(i))
    return x_,y

    # with this function we have symptom and the associated disease into a dataframe

## Natural Language Functions and Classes

In [ ]:
class cleaning_nlp_processes:

    def __init__(self,description):
        self.description=description

    def same_frequency_words_priority(self,word_list):
        final_list=[]
        parts_of_speech = self.parts_of_speech()
        for i in range(0,len(word_list)):
            words = self.lemmatization(parts_of_speech, [word_list[i][0]])
            if len(words)==0 or word_list[i][0]==words[0]:
                # print(words,word_list[i],"Flag=1")
                final_list.append((word_list[i][0],word_list[i][1],1))
            else:
                # print(words,word_list[i]," Flag=0")
                final_list.append((word_list[i][0],word_list[i][1],0))
        return final_list


    def priority_provide(self, stop_words_remove,freq_dict):
        final_priority_list=[]
        # print(freq_dict)
        import operator
        sorted_x = sorted(freq_dict.items(), key=operator.itemgetter(1),reverse=True)
        import collections
        sorted_dict = dict(collections.OrderedDict(sorted_x))
        sorted_list = [(k, v) for k, v in sorted_dict.items()]
        # print(sorted_list)
        splice_index=[]
        for i in range(1,len(sorted_list)):
            if sorted_list[i-1][1]==sorted_list[i][1]:
                continue
            else:
                splice_index.append(i)
        # print(splice_index)
        if len(splice_index)==0:
            final_priority_list=self.same_frequency_words_priority(sorted_list)

        elif len(splice_index)==1:
            list_1=(sorted_list[:splice_index[0]])
            list_2=(sorted_list[splice_index[0]:])
            final_priority_list = self.same_frequency_words_priority(list_1)
            final_priority_list += self.same_frequency_words_priority(list_2)

        else:
            list_1=(sorted_list[:splice_index[0]])
            final_priority_list=self.same_frequency_words_priority(list_1)

            for i in range(1,len(splice_index)):
                list_i=(sorted_list[splice_index[i-1]:splice_index[i]])
                final_priority_list+=self.same_frequency_words_priority(list_i)
            list_end=(sorted_list[splice_index[len(splice_index)-1]:])
            final_priority_list += self.same_frequency_words_priority(list_end)

        # print(final_priority_list)
        sorted_x = sorted(freq_dict.items(), key=operator.itemgetter(1), reverse=True)
        return final_priority_list

    def keywords_formation(self):
        word_arr = self.tokenizing([self.description])
        stopwords_remove = self.stop_words_removal(word_arr)
        # print(stopwords_remove)
        count = self.freq(stopwords_remove)
        final_prority_array=self.priority_provide(stopwords_remove,count)
        # print(final_prority_array)
        return final_prority_array

    def tokenizing(self, parameters):
        # Tokenising..............................................
        data = '';
        for i in parameters:
            data += i + ' '
        data = data.strip()
        # print(data)
        data = data.split(' ')
        # print(data)
        sentence = ''
        for i in data:
            sentence += i+' '
        sentence=sentence.strip()
        word_arr = word_tokenize(sentence.lower())
        #  Here lower case is done to remove more stopwords, but some information is lost
        # print( word_arr)
        return word_arr

    def stop_words_removal(self, word_arr):
        # Stopwords and Punctuations.........................................
        stop_words = stopwords.words('english')
        # Stop Words are present of different languages, for papers of different languages.
        # Cleaning words(removing stopwords and punctuations
        # print(stop_words)
        punctuations = list(string.punctuation)
        # print(string.punctuation)
        stop_words += punctuations
        # print(len(stop_words))
        file_data = []
        with open('../input/ontobasedclassification/stopwords-1', 'r') as f:
            for line in f:
                for word in line.split():
                    file_data.append(word)
        stop_words += file_data
        stop_words = set(stop_words)
        stop_words = list(stop_words)
        new_word_arr = []
        for i in word_arr:
            new_word_arr.append(i.lower())
        clean_words = [w for w in new_word_arr if not w in stop_words]
        # print(len(stop_words))
        # print("After cleaning the stopwords, no of words:", len(clean_words))
        # print(clean_words)
        return clean_words

    def freq(self, words):
        dict = {
        }
        for i in words:
            dict[i] = words.count(i)
        # print("Dictionary after stopwords removal(no lemmatization)")
        # print(dict)
        return dict

    def lemmatization(self, part_of_speech, clean_words):
        lemmatized = []
        lemmatizer = WordNetLemmatizer()
        # print pos[2][0]
        # print clean_words[1]
        for i in range(0, len(part_of_speech)):
            for j in range(0, len(clean_words)):
                if part_of_speech[i][0].lower() == clean_words[j]:
                    # print part_of_speech[i][0], clean_words[j], part_of_speech[i][1]
                    lemmatized.append(
                        (lemmatizer.lemmatize(clean_words[j], pos=self.pos_to_wordnet(part_of_speech[i][1]))).lower())
                    break
                else:
                    continue
        # print("After Lemmatization No. of Words:", len(list(set(lemmatized))))
        # print(lemmatized)
        return list(set(lemmatized))

    def parts_of_speech(self):
        parts_of_speech = pos_tag(word_tokenize(self.description))  # +' '+self.synonym+ ' ' + self.name))
        # print(parts_of_speech)
        return parts_of_speech

    def pos_to_wordnet(self, pos_tag):
        if pos_tag.startswith('J'):
            return wordnet.ADJ
        elif pos_tag.startswith('N'):
            return wordnet.NOUN
        elif pos_tag.startswith('V'):
            return wordnet.VERB
        # elif pos_tag.startswith('M'):
        #     return wordnet.MODAL
        # elif pos_tag.startswith('R'):
        #     return wordnet.ADVERB
        else:
            return wordnet.NOUN

    def stemming(self, cleaning_words):
        '''
        The process of stemmimng is very dumb. Not always give reslutant output. The information passed through it might result in bad output.
        Stemming is the process of finding the root word of the given word.
        Prefer Lemmatization over it.
        '''
        ps = PorterStemmer()
        # stem_words = ['play', 'playing', 'played', 'player', "happy", 'happier']
        stemmed_words = [ps.stem(w) for w in cleaning_words]
        # print(len(stemmed_words))
        # print(stemmed_words)
        return stemmed_words

Let's try the function for the text processing on the dataset

In [ ]:
import csv
with open('../input/ontobasedclassification/Final_Dataset.csv', 'r') as csvFile:
        reader = csv.reader(csvFile)
        additional_keywords = list(reader)
csvFile.close()
    
temp=cleaning_nlp_processes(additional_keywords[4][2]+additional_keywords[4][3])
temp.keywords_formation()

[('internal', 2, 1),
 ('body', 2, 1),
 ('organs', 2, 0),
 ('bowel', 2, 1),
 ('problems', 2, 0),
 ('adhesion', 1, 1),
 ('band', 1, 1),
 ('scar', 1, 1),
 ('tissue', 1, 1),
 ('joins', 1, 0),
 ('surfaces', 1, 0),
 ('connected', 1, 0),
 ('tissues', 1, 0),
 ('stick', 1, 1),
 ('adhere', 1, 1),
 ('surfaces.adhesions', 1, 1),
 ('affect', 1, 1),
 ('female', 1, 1),
 ('reproductive', 1, 1),
 ('ovaries', 1, 0),
 ('fallopian', 1, 1),
 ('tubes', 1, 0),
 ('area', 1, 1),
 ('heart', 1, 1),
 ('spine', 1, 1),
 ('hand', 1, 1),
 ('cause', 1, 1),
 ('range', 1, 1),
 ('including', 1, 0),
 ('infertility', 1, 1),
 ('dyspareunia', 1, 1),
 ('painful', 1, 1),
 ('intercourse', 1, 1),
 ('pelvic', 1, 1),
 ('pain', 1, 1),
 ('obstruction', 1, 1),
 ('blockage', 1, 1),
 ('adhesions', 1, 0),
 ('lead', 1, 1),
 ('complex', 1, 1),
 ('set', 1, 1),
 ('called', 1, 0),
 ('adhesion-related', 1, 1),
 ('disorder', 1, 1),
 ('ard', 1, 1)]

## Ontology Managment Functions

In [ ]:
class keywords_from_ontology:
    def __init__(self,properties):
        # self.defintion=definition
        # self.synonym=synonym
        # self.entity_name=entity_name
        self.properties=properties

    def keywords_creation(self):
        tokens=self.tokenizing(self.properties)
        clean_words=self.stop_words_removal(tokens)
        return clean_words

    def stop_words_removal(self, word_arr):
        # Stopwords and Punctuations.........................................
        stop_words = stopwords.words('english')
        # Stop Words are present of different languages, for papers of different languages.
        # Cleaning words(removing stopwords and punctuations
        # print(stop_words)
        
        punctuations = list(string.punctuation)
        # print(string.punctuation)
        stop_words += punctuations
        # print(len(stop_words))
        file_data = []
        with open('../input/ontobasedclassification/stopwords-1', 'r') as f:
            for line in f:
                for word in line.split():
                    file_data.append(word)
        stop_words += file_data
        stop_words = set(stop_words)
        stop_words = list(stop_words)
        new_word_arr = []
        for i in word_arr:
            new_word_arr.append(i.lower())
        clean_words = [w for w in new_word_arr if not w in stop_words]
        # print(len(stop_words))
        # print("After cleaning the stopwords, no of words:", len(clean_words))
        # print(clean_words)
        return clean_words

    def tokenizing(self, parameters):
        # Tokenising..............................................
        data = '';
        for i in parameters:
            data += i + ' '
        data = data.strip()
        # print(data)
        data = data.split(' ')
        # print(data)
        sentence = ''
        for i in data:
            sentence += i+' '
        sentence=sentence.strip()
        word_arr = word_tokenize(sentence.lower())
        #  Here lower case is done to remove more stopwords, but some information is lost
        # print( word_arr)
        return word_arr

class Node:
    def __init__(self,entity,keywords):
        self.entity=entity
        # self.children=children
        self.keywords=keywords
        # self.deprecated=deprecated

def ontology_creation():
    with open('../input/ontobasedclassification/DOID_Ontology - Sheet1.csv', 'r') as csvFile:
        reader = csv.reader(csvFile)
        data = list(reader)
    csvFile.close()
    # for i in range(0,len(data[0])):
    #   print(i,data[0][i])
    # print(additional_keywords[0])
    temp_object_for_keywords = keywords_from_ontology('')
    nodes=[]
    for k in range(1,len(data)):
      parent_keywords=[]
      child_keywords=[]
      temp_object_for_keywords = keywords_from_ontology([str(data[k][0]), str(data[k][3]),
                                                               str(data[k][4]),str(data[k][5]),
                                                               str(data[k][6]), str(data[k][7]),
                                                               str(data[k][7]),str(data[k][9]), 
                                                               str(data[k][10]),str(data[k][13]),
                                                               str(data[k][14]),str(data[k][24]),
                                                               str(data[k][25]),str(data[k][54]),
                                                               str(data[k][55]),str(data[k][58])])      
      for i in range(1,len(data)):
        if data[i][0]==data[k][2]:
          temp_object_for_keywords_parent = keywords_from_ontology([str(data[i][0]), str(data[i][3]),
                                                               str(data[i][4]),str(data[i][5]),
                                                               str(data[i][6]), str(data[i][7]),
                                                               str(data[i][8]),str(data[i][9]), 
                                                               str(data[i][10]),str(data[i][13]),
                                                               str(data[i][14]),str(data[i][24]),
                                                               str(data[i][25]),str(data[i][54]),
                                                               str(data[i][55]),str(data[i][58])])
          parent_keywords=list(set(temp_object_for_keywords_parent.keywords_creation()))
          break
        else:
          continue

      child_keywords=list(set(temp_object_for_keywords.keywords_creation()))
      obj=Node(str(data[k][0]),list(set(child_keywords+parent_keywords)))
      nodes.append(obj)
    
    return nodes

def ontology_mathing(tree,keywords):
    # print('Keywords formed from given x data:',keywords)
    # print()
    nodes_to_search=[]
    found_nodes = []
    priority_1 = []
    priority_2 = []
    priority_3 = []

    for i in range(0, len(keywords)):
        if keywords[i][1] != 1:
            priority_1.append(keywords[i][0])
        else:
            if keywords[i][2] == 1:
                priority_2.append(keywords[i][0])
            else:
                priority_3.append(keywords[i][0])

    # nodes_to_search.append(tree[0])
    # nodes_to_search.append(tree[76])
    # i=0
    # flag=0
    # while i in range(0,len(nodes_to_search)):
    #     if i==0:
    #         node=nodes_to_search.index(tree[0])
    #         print('Entity whose children are Currently Under Search:',tree[node].entity)
    #         flag=0
    #     else:
    #         node = tree.index(nodes_to_search[i])
    #         print('Entity whose children are Currently Under Search:', tree[node].entity)
    #         flag=1
    #     for j in range(0,len(tree[node].children)):
    #         print('Child under Consideration:',tree[node].children[j])
    #         for k in range(i,len(tree)):
    #             if tree[k].entity==tree[node].children[j]:
    #                 print('Found Entity in Tree:', tree[k].entity)
    #                 if flag==0:
    #                     for l in range(0,len(keywords)):
    #                         if keywords[l][0] in tree[k].keywords:
    #                             found_nodes.append((k,tree[k].entity,keywords[l][0]))
    #                             nodes_to_search.append(tree[k])
    #                             print('Keyword ',keywords[l][0],' Found in Node:',tree[k].entity)
    #                             break
    #                         else:
    #                             continue
    #                 else:
    #                     nodes_to_search.append(tree[k])
    #                     for l in range(0,len(keywords)):
    #                         if keywords[l][0] in tree[k].keywords:
    #                             found_nodes.append((k,tree[k].entity,keywords[l][0]))
    #                             print('Keyword ',keywords[l][0],' Found in Node:',tree[k].entity)
    #                             break
    #                         else:
    #                             continue
    #             else:
    #                 continue
    #     i=i+1
        # print(nodes_to_search)

    for i in range(0,len(tree)):
        priority_1_count = 0
        priority_2_count = 0
        priority_3_count = 0
        for j in range(0,len(priority_1)):
            if priority_1[j].lower() in [x.lower() for x in tree[i].keywords]:
                priority_1_count = priority_1_count + 1
            else:
                continue
        for j in range(0, len(priority_2)):
            if priority_2[j].lower() in [x.lower() for x in tree[i].keywords]:
                priority_2_count = priority_2_count + 1
            else:
                continue

        for j in range(0, len(priority_3)):
            if priority_3[j].lower() in [x.lower() for x in tree[i].keywords]:
                priority_3_count = priority_3_count + 1
            else:
                continue

        # print("Priority_1:", priority_1_count)
        # print("Priority_2:", priority_2_count)
        # print("Priority_3:", priority_3_count)

        # if priority_1_count>=1:
        #     if priority_2_count>=1:
        #         if priority_3_count>=1:
        #             found_nodes.append((tree[i].entity))
        #         else:
        #             continue
        #     else:
        #         continue
        # else:
        #     continue
        if priority_1_count+priority_2_count+priority_3_count>=1:
            # print('Possible Node :', tree[i].entity)
            # print('Possible Node Keywords:', tree[i].keywords)
            # print('Keywords Passed:',keywords)
            found_nodes.append((tree[i].entity))
        else:
            continue

    # for i in range(0,len(found_nodes)):
    #     print('Possible Class/Node:',found_nodes[i])
    classes=[]
    for i in range(0,len(found_nodes)):
        classes.append(found_nodes[i])
    classes=list(set(classes))
    # print(classes)
    return classes

# Classification Models Functions

## Classification without the Ontology

In [ ]:
# ------CLASSIFICATION PROCESS DEFINITION WITHOUT ONTOLOGY APPROACH---------
def Classification_without_Ontology(x_train, y_train,x_test,y_test):
    names=['Multinomial Naive Bayes','Bagging', 'KNN', 'SVM','Decision Tree','Random Forest']#,'Gradient Boost','Logistic Regression']
    count_vector = CountVectorizer(max_features=400)
    training_data = count_vector.fit_transform(x_train)
    predictions=[]
    testing_data = count_vector.transform(x_test)
 
    naive_bayes = MultinomialNB()
    grad_boost=GradientBoostingClassifier()
    bagging=BaggingClassifier()
    KNN=KNeighborsClassifier(n_neighbors=2)
    SVM=LinearSVC()
    decision_tree=DecisionTreeClassifier()
    random_forest=RandomForestClassifier()
    #lr=LogisticRegression()
    
    classifiers=[naive_bayes,bagging,KNN,SVM,decision_tree,random_forest]#,grad_boost,lr]
    for i in range(0,len(classifiers)):
      # DEBUG PRINTING
      print('Classifier in use:'+names[i])
      classifiers[i].fit(training_data, y_train)
      # DEBUG PRINTING
      print('Done with', names[i])
    # print(count_vector.get_feature_names())
      predictions.append(classifiers[i].predict(testing_data))
    
    return predictions

# ------CLASSIFICATION PROCESS EVALUATION WITHOUT ONTOLOGY APPROACH---------
def entire_process_without_ontology():
    data_x,data_y=dataset_creation()
    names=['Multinomial Naive Bayes','Bagging', 'KNN', 'SVM','Decision Tree','Random Forest']#,'Gradient Boost']#,'Logistic Regression']
    x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2)
    predictions_without_ontology = Classification_without_Ontology(x_train[:500], y_train[:500],x_test[:100],y_test[:100])
    for i in range(0,len(predictions_without_ontology)):
      print('Classifier under consideration: '+ names[i])
      print('Accuracy score:',format(accuracy_score(y_test[:len(predictions_without_ontology[i])], predictions_without_ontology[i])))
      print('Precision score:',format(precision_score(y_test[:len(predictions_without_ontology[i])], predictions_without_ontology[i], average='micro')))
      print('Recall score:',format(recall_score(y_test[:len(predictions_without_ontology[i])], predictions_without_ontology[i], average='micro')))
      print('F1 score:',format(f1_score(y_test[:len(predictions_without_ontology[i])], predictions_without_ontology[i], average='micro')))

In [ ]:
entire_process_without_ontology()

Classifier in use:Multinomial Naive Bayes
Done with Multinomial Naive Bayes
Classifier in use:Bagging
Done with Bagging
Classifier in use:KNN
Done with KNN
Classifier in use:SVM
Done with SVM
Classifier in use:Decision Tree
Done with Decision Tree
Classifier in use:Random Forest
Done with Random Forest
Classifier under consideration: Multinomial Naive Bayes
Accuracy score: 0.49
Precision score: 0.49
Recall score: 0.49
F1 score: 0.49
Classifier under consideration: Bagging
Accuracy score: 0.33
Precision score: 0.33
Recall score: 0.33
F1 score: 0.33
Classifier under consideration: KNN
Accuracy score: 0.3
Precision score: 0.3
Recall score: 0.3
F1 score: 0.3
Classifier under consideration: SVM
Accuracy score: 0.6
Precision score: 0.6
Recall score: 0.6
F1 score: 0.6
Classifier under consideration: Decision Tree
Accuracy score: 0.29
Precision score: 0.29
Recall score: 0.29
F1 score: 0.29
Classifier under consideration: Random Forest
Accuracy score: 0.42
Precision score: 0.42
Recall score: 0.

## Classification with the Ontology

In [ ]:
# ------CLASSIFICATION PROCESS DEFINITION WITH ONTOLOGY APPROACH---------
def Classification_with_Ontology(x_train, y_train,x_test,y_test):
    ontology_tree=ontology_creation()
    tree=ontology_tree
    predictions=[[],[],[],[],[],[],[]]
    c1,c2,c3=0,0,0
    all_classes = list(set(list(set(y_train))+list(set(y_test))))
    names=['Multinomial Naive Bayes','Bagging', 'KNN', 'SVM','Decision Tree','Random Forest','Logistic Regression']
    keywords_for_matching=[]
    
    with open('../input/ontobasedclassification/Final_Dataset.csv', 'r') as csvFile:
        reader = csv.reader(csvFile)
        additional_keywords = list(reader)
    csvFile.close()
    
    for i in range(1,len(additional_keywords)):
      obj = cleaning_nlp_processes(additional_keywords[i][2]+additional_keywords[i][3])
      additional_words = obj.keywords_formation()
      # print('Keywords From Dataset for '+additional_keywords[i][0] +' are:')
      # print(additional_words)
      keywords_for_matching.append((additional_keywords[i][0],additional_words))

    count_vector_entire_data_set = CountVectorizer(max_features=400)
    training_data_entire_data_set = count_vector_entire_data_set.fit_transform(x_train)

    naive_bayes_entire_data_set = MultinomialNB()
    bagging_entire_data_set=BaggingClassifier()
    KNN_entire_data_set=KNeighborsClassifier(n_neighbors=2)
    SVM_entire_data_set=LinearSVC()
    decision_tree_entire_data_set=DecisionTreeClassifier()
    random_forest_entire_data_set=RandomForestClassifier()
    lr_entire_dataset=LogisticRegression()
    
    classifiers_entire_data_set=[naive_bayes_entire_data_set,bagging_entire_data_set,KNN_entire_data_set,SVM_entire_data_set,decision_tree_entire_data_set,
                                 random_forest_entire_data_set,lr_entire_dataset]
    
    naive_bayes = MultinomialNB()
    bagging=BaggingClassifier()
    KNN=KNeighborsClassifier(n_neighbors=2)
    SVM=LinearSVC()
    decision_tree=DecisionTreeClassifier()
    random_forest=RandomForestClassifier()
    lr=LogisticRegression()

    classifiers=[naive_bayes,bagging,KNN,SVM,decision_tree,random_forest,lr]
    
    for i in range(0,len(classifiers_entire_data_set)):
      # print('Classifier in use:'+names[i])
      classifiers_entire_data_set[i].fit(training_data_entire_data_set, y_train)
    # print(count_vector.get_feature_names())
    
    for z in range(0,len(x_test)):
      # print('Disease Under Consideration:',y_test[z])
      obj = cleaning_nlp_processes(x_test[z])
      words = obj.keywords_formation()
      # print('Initially Keywords from symtoms:')
      # print(words)
      for q in range(0,len(keywords_for_matching)):
        # print(y_test[z],keywords_for_matching[q][0],y_test[z].strip()==keywords_for_matching[q][0].strip())
        if y_test[z].strip()==keywords_for_matching[q][0].strip():
          words=words+keywords_for_matching[q][1]
          break
        else:
          continue
      # print('Adding additional Keywords From Dataset are:')
      # print(words)
      # flag=1
      # for i in range(0,len(tree)):
      #   if tree[i].entity[len(tree[i].entity)-1]=="'":
      #     # print(tree[i].entity[0:len(tree[i].entity)-2].strip(),data_y[no])
      #     if set(word_tokenize(y_test[z])).issubset(word_tokenize(tree[i].entity[0:len(tree[i].entity)-2])):
      #       print('Ontology Node:',tree[i].entity[0:len(tree[i].entity)-2])
      #       print('Keywords of ontology Node:',tree[i].keywords)
      #       flag=0
      #       break
      #     else:
      #       continue
      #   else:      
      #     # print(tree[i].entity,data_y[no])
      #     if set(word_tokenize(y_test[z])).issubset(word_tokenize(tree[i].entity)):
      #       print('Ontology Node:',tree[i].entity)
      #       print('Keywords of ontology Node:',tree[i].keywords)
      #       flag=0
      #       break
      #     else:
      #       continue
      common_classes=[]
      # if flag==1:
        # print('Disease doesn"t occur in ontology')
        # pass
      possible_classes=ontology_mathing(ontology_tree,words)
      # print('Possibilities After Ontology Matching:',possible_classes)
      for i in range(0,len(all_classes)):
          dataset_elements=[x.lower() for x in list(set(word_tokenize(all_classes[i])))]
          for j in range(0,len(possible_classes)):
            ontology_node=[x.lower() for x in list(set(word_tokenize(possible_classes[j])))]
            if set(dataset_elements).issubset(set(ontology_node)):
              common_classes.append(all_classes[i])
            else:
              continue      
      common_classes=list(set(common_classes))
      # print('Final Possibilites After Ontology Dataset Intersections:',common_classes)
      # print('Number of possible Classes after intersection:',len(common_classes))
      # print('Does Actual Class exsist in Possible Classes after intersection:', y_test[z] in common_classes)
      if len(common_classes)==0:
        c1=c1+1
        # print('No Entry Found in Dataset and Ontology, Using Normal Classification.........')
        # print('Comparing Predictions...............')
        # print('Actual Value:',y_test[z])
        testing_data = count_vector_entire_data_set.transform([x_test[z]])[0]
        for h in range(0,len(classifiers_entire_data_set)):
          prediction= classifiers_entire_data_set[h].predict(testing_data)[0] 
          predictions[h].append(prediction)
          # print('Comparing Predictions...............')
          # print('Classifier in use: ',names[h])
          # print('Actual value: ',y_test[z])
          # print('Predicted Value:',prediction)  
      else:
        c2=c2+1
        # print('Using Ontology Matching............')
        indexes_to_use = []
        for j in range(0, len(y_train)):
          if y_train[j] in common_classes:
            indexes_to_use.append(j)
          else:
            continue
        new_x=[x_train[x] for x in indexes_to_use] #qua sto solamente prendendo dal train solo le classi che ho trovato in comune con l'ontologia
        new_y=[y_train[x] for x in indexes_to_use] # sia x che y

        count_vector = CountVectorizer(max_features=400)
        training_data = count_vector.fit_transform(new_x)
        testing_data = count_vector.transform([x_test[z]])[0]
        
        for h in range(0,len(classifiers)):
          classifiers[h].fit(training_data,new_y)
          prediction= classifiers[h].predict(testing_data)[0] 
          predictions[h].append(prediction)
          # print('Comparing Predictions...............')
          # print('Classifier in use: ',names[h])
          # print('Actual Value:',y_test[z])
          # print('Predicted Value:',prediction)
        # if set(word_tokenize(y_test[z])).issubset(set(word_tokenize(prediction))):
          # c3=c3+1
      # print(c1,c2,c3)
      print(((z+1)/len(x_test))*100,'% Completed')
      # print()
    return predictions

def entire_process_with_ontology():
    names=['Multinomial Naive Bayes','Bagging', 'KNN', 'SVM','Decision Tree','Random Forest','Logistic Regression']#'Gradient Boost']
    data_x,data_y=dataset_creation()
    x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2,random_state=0)
    predictions_with_ontology = Classification_with_Ontology(x_train[:500],y_train[:500],x_test[:100] ,y_test[:100])
    print(predictions_with_ontology)

    for i in range(0,len(predictions_with_ontology)):
      print('Classifier under consideration:'+ names[i])
      print('Accuracy score:',format(accuracy_score(y_test[:len(predictions_with_ontology[i])], predictions_with_ontology[i])))
      print('Precision score:',format(precision_score(y_test[:len(predictions_with_ontology[i])], predictions_with_ontology[i], average='micro')))
      print('Recall score:',format(recall_score(y_test[:len(predictions_with_ontology[i])], predictions_with_ontology[i], average='micro')))
      print('F1 score:',format(f1_score(y_test[:len(predictions_with_ontology[i])], predictions_with_ontology[i], average='micro')))

In [ ]:
entire_process_with_ontology()

1.0 % Completed
2.0 % Completed
3.0 % Completed
4.0 % Completed
5.0 % Completed
6.0 % Completed
7.000000000000001 % Completed
8.0 % Completed
9.0 % Completed
10.0 % Completed
11.0 % Completed
12.0 % Completed
13.0 % Completed
14.000000000000002 % Completed
15.0 % Completed
16.0 % Completed
17.0 % Completed
18.0 % Completed
19.0 % Completed
20.0 % Completed
21.0 % Completed
22.0 % Completed
23.0 % Completed
24.0 % Completed
25.0 % Completed
26.0 % Completed
27.0 % Completed
28.000000000000004 % Completed
28.999999999999996 % Completed
30.0 % Completed
31.0 % Completed
32.0 % Completed
33.0 % Completed
34.0 % Completed
35.0 % Completed
36.0 % Completed
37.0 % Completed
38.0 % Completed
39.0 % Completed
40.0 % Completed
41.0 % Completed
42.0 % Completed
43.0 % Completed
44.0 % Completed
45.0 % Completed
46.0 % Completed
47.0 % Completed
48.0 % Completed
49.0 % Completed
50.0 % Completed
51.0 % Completed
52.0 % Completed
53.0 % Completed
54.0 % Completed
55.00000000000001 % Completed
56.00

## Testing Functions

### Dataset Analysis Function

In [ ]:
def data_analysis():
    total_count=0
    c=0
    print('Analysing Common Diseases in Dataset and Ontology.................')
    #data_x,data_y=dataset_creation()
    #ontology_tree = ontology_creation()
    for i in list(set(data_y)):
      print('Disease From Dataset:',i)
      dataset_elements=[x.lower() for x in list(set(word_tokenize(i)))]
      for j in range(0,len(ontology_tree)):
            ontology_node=[x.lower() for x in list(set(word_tokenize(ontology_tree[j].entity)))]
            if set(dataset_elements).issubset(set(ontology_node)):
                c=c+1
                # DEBUG PRINTING
                print('Disease Name Ontology from ontology:', ontology_tree[j].entity)
                # indices = [l for l, x in enumerate(data_y) if x == str(ontology_tree[i].entity)]
                # print('Indices in Dataset for :',i, ' are:', indices)
                obj = cleaning_nlp_processes(data_x[random.choice(indices)])
                words = obj.keywords_formation()
                print('Keywords From Dataset are:')
                print(words)
                obj = cleaning_nlp_processes(data_x[random.choice(indices)])
                words = obj.keywords_formation()
                print('Keywords From Dataset are:')
                print(words)
                obj = cleaning_nlp_processes(data_x[random.choice(indices)])
                words = obj.keywords_formation()
                print('Keywords From Dataset are:')
                print(words)
                obj = cleaning_nlp_processes(data_x[random.choice(indices)])
                words = obj.keywords_formation()
                print('Keywords From Dataset are:')
                print(words)
                print('Ontology Information:')
                print('Keywords From Ontology Node:')
                print(ontology_tree[j].keywords)
                print('Children of Ontology Node:',ontology_tree[i].children)
                # END DEBUG PHASE
            else:
                continue
      print('Number of Nodes Match for this disease are:',c)
      if c!=0:
        total_count=total_count+1
      else:
        print("This Disease can't matched in the ontology:",i)
      c=0
    print(total_count)

In [ ]:
data_analysis()

# Save the notebook in PDF

In [ ]:
!apt update
!apt install texlive-xetex texlive-fonts-recommended texlive-generic-recommended

import re, pathlib, shutil

# Get a list of all your Notebooks
notebooks = [x for x in pathlib.Path("/content/drive/My Drive/Colab Notebooks").iterdir() if 
             re.search(r"\.ipynb", x.name, flags = re.I)]

for i, n in enumerate(notebooks):
    print(f"\nProcessing  [{i+1:{len(str(len(notebooks)))}d}/{len(notebooks)}]  {n.name}\n")

    # Optionally copy your notebooks from gdrive to your vm
    shutil.copy(n, n.name)
    n = pathlib.Path(n.name)

!jupyter nbconvert "{n.as_posix()}" --to pdf --output "{n.stem.replace(" ", "_")}"

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Building dependency tree       
Reading state infor